# Тестовое задание Стажёр в команду CoreML
### Маслов Михаил

In [1]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from pprint import pprint

In [2]:
import os

DATA_DIR = r'/home/mika/JupyterNotebooks/VkTest/data/'
os.chdir(DATA_DIR)

# использую для всех случайных процессов чтобы можно было воспроизвести результаты
RNG_SEED = 42

# для встроенного распараллеливания библиотеки, не уверен что работает
os.environ['MKL_THREADING_LAYER'] = 'tbb'
os.environ['LK_NUM_PROCS'] = '8,4'
os.environ['NUMBA_NUM_THREADS'] = '6'

In [3]:
rating = pd.read_csv('rating.csv')

In [4]:
print('rating shape:\t', rating.shape)
print('min rating:\t',rating['rating'].min())
print('max rating:\t',rating['rating'].max())
print('uniq users count:', rating['userId'].unique().size)
print('uniq movies count:', rating['movieId'].unique().size)
rating.head()

rating shape:	 (20000263, 4)
min rating:	 0.5
max rating:	 5.0
uniq users count: 138493
uniq movies count: 26744


,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


Также стоит отметить, что распределение количества информации известной о пользователях - неравномерное, <br>оно похоже на распределние <a href="https://en.wikipedia.org/wiki/Power_law">степенного закона</a>.<br>
Чтобы не загромождать ноутбук графиками, я хотел бы сослаться на неплохую <a href='https://www.kaggle.com/code/saadmuhammad17/data-analysis-of-movielens-25m-dataset'>визуализацию</a> датасета с kaggle.

## Eval metrics: RMSE & NDCG

<p style="font-size: 20">
Обычно для измерения точности рекомнедации используют <i>RMSE</i>, как например в <i>Netflix Prize</i>.<br> 
Однако достижение хороших показателей с точки зрения <i>RMSE</i> не всегда гарантирует хорошие показатели рекомендательной системы.<br> 
Также можно добавить, что рейтинг это порядковые данные, то есть 5-3 != 3-1, 
а "<i>чистый</i>" <i>RMSE</i> не учитывет не линейность рейтингов<br><br>
Поэтому для измерения качества рекомендательной системы мы также будем использовать метрику <i>NDCG</i>, <br>которая позволит нам оценить соответсвие "<i>идеальной</i>" рекомендации.
</p>

## Split train validate

переписать мб

Так как наша задача это предсказываение рейтинга фильмов для пользователей, то мы разобьём датасет для каждого пользователя. <br> Также при разбиение мы учтём время, хоть и не все модели его использует, но логично предположить, <br>что время это полезный признак имеющий смысл. <br>
Я взял 5 разбиений для кросс валидации, потому что это наиболее популярный варинат и увеличение или умененьшение этого числа не должно дать существенных изменений.<br>
Также я взял 5 последних фильмов у каждого пользователя для валидации, так как минимальное колчиество оценок у пользователя 20 и, как мне кажется, при оценки ранжироврованого списка фильмов оценка бует объективнее, если мы будем угадывать для всех одинаковое число фильмов.

In [10]:
import lenskit.crossfold as xf

N_SPLITS = 5

rating = rating.rename(columns={'userId': 'user', 'movieId': 'item'})
for i, tp in enumerate(xf.partition_users(rating, N_SPLITS, xf.LastN(5), rng_spec=RNG_SEED)):
    tp.train.to_csv('20m.train-%d.csv' % (i,))
    tp.test.to_csv('20m.test-%d.csv' % (i,))

## Collaborative filtering

### Метод LFM (SVD-like <a href="https://sifter.org/~simon/journal/20061211.html">FunkSVD</a>)

Алгоритм:
<p>
Данный метод использует векторное представление пользователя и объекта,<br>
а также средний рейтинг пользователя и объекта<br>
С помощью градиентного спуска мы находим векторы для каждого пользователя и объекта.<br>
Важной частью этого алгоритма является L2-регуляризация, она предотварщает модель от переобучения, что является проблемой SVD++. Можно отметить, что L1-регуляризация не даёт качественого прироста в точности
</p>
<p>
    
Гиперпараметрами алгоритма являются:
- количество эпох и/или эпсилон изменения ошибки
- количество признаков для предстваления пользователя и объекта

</p>
<img src='imgs/svd.png'alt="без регуляризации">
<i style="float:right;">без регуляризации</i>
<img src='imgs/funksvd.png'>
<i style="float:right;">с регуляризацией</i>
<br>


Далее прочитав статью <a href="https://sifter.org/~simon/journal/20061211.html">Simon Funk</a> и проанализировав <a href="https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data">датасет</a> с <i>Netflix Prize</i>, я решил что наши данные очень схожи и <br> поэтому можно взять гиперпараметры из блога призёра этого исторического конкурса.<br><br>
А именно:
- количество эпох 120
- количество признаков 40

In [ ]:
%%time
from lenskit.algorithms.funksvd import FunkSVD
from lenskit.metrics.predict import rmse, global_metric
from lenskit.topn import ndcg
from joblib import Parallel, delayed

results = []

N_SPLITS = 5
n_features = 40
iterations = 120

def train_test_eval_svd(i):
    cf_train = pd.read_csv('20m.train-%d.csv' % (i,))
    cf_test =  pd.read_csv('20m.test-%d.csv' % (i,))
    
    cf_model_svd = FunkSVD(features=n_features, iterations=iterations, range=(0.5,5))
    cf_model_svd.fit(cf_train)
    
    # Предсказываем
    cf_pred = cf_model_svd.predict(cf_test)
    cf_test['prediction'] = cf_pred

    # Оцениваем
    cf_model_ndcg = ndcg(cf_test.rename(columns={'rating': 'original_rating','prediction': 'rating'}), cf_test)
    cf_model_rmse = global_metric(cf_test, metric=rmse)

    result = {
        'n_features': n_features,
        'n_epochs': iterations,
        'rmse': cf_model_rmse,
        'ndcg': cf_model_ndcg,
    }
    
    return result

# работа с процессами для библиотеки
current_process = psutil.Process()
subproc_before = set([p.pid for p in current_process.children(recursive=True)])

# на больше n_jobs ОЗУ не хватает
results = Parallel(n_jobs=3, backend='multiprocessing')(
    delayed(train_test_eval_svd)(i) for i in tqdm(range(N_SPLITS)))

# особенность библиотеки чтобы завершить выполнение
subproc_after = set([p.pid for p in psutil.Process().children(recursive=True)])
for subproc in subproc_after - subproc_before:
    psutil.Process(subproc).terminate()


In [32]:
avg_rmse = sum([res['rmse'] for res in results])/len(results)
avg_ndcg = sum([res['ndcg'] for res in results])/len(results)
print('Avg RMSE:', avg_rmse)
print('Avg NDCG:', avg_ndcg)

Avg RMSE: 0.8573221431188687
Avg NDCG: 0.9694998266280797


## Collaborative + content filtering

In [195]:
%reset -f

# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from pprint import pprint
import os
from datetime import datetime

In [204]:
DATA_DIR = r'/home/mika/JupyterNotebooks/VkTest/data/'
os.chdir(DATA_DIR)

# использую для всех случайных процессов чтобы можно было воспроизвести результаты
RNG_SEED = 42

genome_scores = pd.read_csv('genome_scores.csv')
genome_tags = pd.read_csv('genome_tags.csv')
link = pd.read_csv('link.csv')
movie = pd.read_csv('movie.csv')
rating = pd.read_csv('rating.csv')
tag = pd.read_csv('tag.csv')

n_features = 40

In [205]:
display('genome_scores', genome_scores.head())
display('genome_tags', genome_tags.head())
display('movie', movie.head())
display('rating', rating.head())
display('tag', tag.head())

'genome_scores'

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


'genome_tags'

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


'movie'

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


'rating'

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


'tag'

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


### Категориальные признаки

Категориальный признак - это признак, который не выражает непрерывную величину, а принимает одно из фиксированных значений.<br>

Этот признак можно первратить в многомерный вектор, чтобы его можно было использовать в модели.<br>

### Нормализация

Ещё можно нормализовать время(timestamp), тк оно слишком велико и лежит в определённых рамках. 

### Текстовые данные

Также можно добавить текстовые признаки в нашу модель. Обычно такие вещи, как описания продуктов, представляют собой текст в свободной форме, и мы можем надеяться, что наша модель сможет научиться использовать содержащуюся в них информацию для выработки лучших рекомендаций, особенно в случае холодного запуска или длинного хвоста.

<!-- ### TODO: Убрать мешуру и всё в одну модель, поменять tsdf где можно(медленные) -->

In [206]:
import tensorflow as tf

rating['timestamp'] = rating['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').toordinal())
rating['userId'] = rating['userId'].to_numpy().astype(str)
ratings = tf.data.Dataset.from_tensor_slices(dict(rating.merge(movie)))
movies = tf.data.Dataset.from_tensor_slices(dict(movie))


ratings = ratings.map(lambda x: {
    "movie_title": x["title"],
    "user_id": x["userId"],
    "timestamp": x["timestamp"],
})
movies = movies.map(lambda x: x["title"])

In [ ]:
timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

# unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
# unique_user_ids = np.asarray(np.unique(np.concatenate(list(ratings.batch(1_000).map(lambda x: x["user_id"]))))).astype('float32')
unique_movie_titles = movie.tittle.unique()
unique_user_ids = rating.userId.unique().astype(str)

### Собираем итоговую модель

In [ ]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])
    self.timestamp_embedding = tf.keras.Sequential([
        tf.keras.layers.Discretization(timestamp_buckets.tolist()),
        tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32),
    ])
    self.normalized_timestamp = tf.keras.layers.Normalization(
        axis=None
    )

    self.normalized_timestamp.adapt(timestamps)

  def call(self, inputs):
    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["timestamp"]),
        tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1)),
    ], axis=1)

In [ ]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.title_vectorizer.adapt(movies)

  def call(self, titles):
    return tf.concat([
        self.title_embedding(titles),
        self.title_text_embedding(titles),
    ], axis=1)

In [ ]:
class CandidateModel(tf.keras.Model):

  def __init__(self, layer_sizes):
    super().__init__()

    self.embedding_model = MovieModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [ ]:
class QueryModel(tf.keras.Model):

  def __init__(self, layer_sizes):
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = UserModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [ ]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, layer_sizes):
    super().__init__()
    self.query_model = QueryModel(layer_sizes)
    self.candidate_model = CandidateModel(layer_sizes)
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "timestamp": features["timestamp"],
    })
    movie_embeddings = self.candidate_model(features["movie_title"])

    return self.task(
        query_embeddings, movie_embeddings, compute_metrics=not training)

In [ ]:
tf.random.set_seed(RNG_SEED)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [ ]:
num_epochs = 100

model = MovielensModel([32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

one_layer_history = model.fit(
    cached_train,
    validation_data=cached_test,
    validation_freq=5,
    epochs=num_epochs,
    verbose=0)

Последняя модель явно вышла, сырой, но это только из-за моих технических проблем. Возможно было бы чуть больше времени, я бы сделал по-другому и красивее